In [72]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

import datetime


site_ec_dict = {'google': EC.element_to_be_clickable((By.XPATH, "//div[@class='QS5gu sy4vM']")),
                'nasdaq': EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))}

nav_nasdaq_ec_dict = {'select_date': EC.element_to_be_clickable((By.XPATH,
                            "//button[@class='date-picker__toggle' and @aria-label='click show datepicker popup']")),      
                      'select_searchbox':  EC.visibility_of_element_located((By.CLASS_NAME, "date-picker__input")),
                      'apply_data_button': EC.element_to_be_clickable((By.XPATH,
                            "//button[@class='date-picker__apply']"))}


#EC.element_to_be_clickable((By.ID, "date-picker__toggle"))}

def accept_cookies(site = 'google'):    
    try:
        accept = WebDriverWait(driver, 1).until(site_ec_dict[site])
        accept.click()
    except:
        print("No 'Accept' button found or unable to click.")


def google(query):
    # Open the website
    driver.get("https://google.com")
    accept_cookies('google')
    search_box = driver.find_element(By.NAME, "q")

     # Enter your search query
    search_query = query
    search_box.send_keys(search_query)

    # Simulate hitting Enter
    search_box.send_keys(Keys.RETURN)



def click_site(site):
    driver.find_element(By.XPATH, f"//a[contains(@href, '{site}')]").click()

def select_date(year, month, day, site = 'nasdaq'):
    if site == 'nasdaq':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['select_date']).click()
        driver.execute_script(f"document.querySelector('.date-picker__input').value = '{date}';")
        WebDriverWait(driver, 10).until(nav_nasdaq_ec_dict['apply_data_button']).click()




In [19]:
driver = webdriver.Chrome()

google('nasdaq earnings calendar')
click_site('nasdaq.com')



In [20]:
accept_cookies('nasdaq')

In [74]:
year, month, day = 2024, 3, 5
select_date(year, month, day, site = 'nasdaq')